In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json
import os
from datetime import datetime
from pprint import pprint

import database
import helper
import utils

In [ ]:
LOG = utils.CustomLogger("CustomLogger", log_level= "info", display_loglevel= False, display_datetime= False)
PICKLE_LIB = utils.PickleLib(data_path="./data", logger= LOG)

# Data Search

In [ ]:
client = database.connect()
db = client.JiraRepos

print(db.list_collection_names())
collection = db['Jira']

In [ ]:
# results = collection.aggregate([
#     {
#         "$match": {
#             "fields.description": {
#                 "$exists": True, 
#                 "$type": "string",
#                 "$regex": ".*During an in-app update.*"
#             }
#         }
#     },
#     {
#         "$sample": {
#             "size": 5
#         }
#     }
# ])


# list = []

# for document in results:
#     list.append(document)
#     print("Id: " + str(document['id']) + " Length: " + str(len(document['fields']['summary'])) + ": " + document['fields']['summary'])

#### Summary

In [ ]:
def getInRange():
    results = collection.aggregate([
        {
            "$match": {
                "fields.summary": {
                    "$exists": True, 
                    "$type": "string"
                },
                "$expr": {
                    "$and": [
                        {"$lt": [{"$strLenCP": "$fields.summary"}, 70]},
                        {"$gt": [{"$strLenCP": "$fields.summary"}, 39]}
                    ]
                }
            }
        },
        {
            "$sample": {
                "size": 5
            }
        }
    ])
    return results

def getShorter():
    results = collection.aggregate([
        {
            "$match": {
                "fields.summary": {
                    "$exists": True, 
                    "$type": "string"
                },
                "$expr": {
                    "$and": [
                        {"$lt": [{"$strLenCP": "$fields.summary"}, 39]}
                    ]
                }
            }
        },
        {
            "$sample": {
                "size": 5
            }
        }
    ])
    return results

def getLonger():
    results = collection.aggregate([
        {
            "$match": {
                "fields.summary": {
                    "$exists": True, 
                    "$type": "string"
                },
                "$expr": {
                    "$and": [
                        {"$gt": [{"$strLenCP": "$fields.summary"}, 70]}
                    ]
                }
            }
        },
        {
            "$sample": {
                "size": 10
            }
        }
    ])
    return results

In [ ]:
summary = []

# results = getInRange()
# results = getShorter()
results = getLonger()

for document in results:
    summary.append(document)
    print("Id: " + str(document['id']) + " Length: " + str(len(document['fields']['summary'])) + ": " + document['fields']['summary'])

#### Arbitrary Structure

In [ ]:
def findKeysByCode(data, target_code):
    matching_keys = []
    for key, value in data.items():
        if 'code' in value and value['code'] == target_code:
            matching_keys.append(key)
    return matching_keys

In [ ]:
with open('issueTypeMapping.json') as f:
    mappedIssueTypes = json.load(f)

In [ ]:
mappedTypes = findKeysByCode(mappedIssueTypes['Jira'], 'Bug Report')
mappedTypes

In [ ]:
tickets = []
# results = collection.aggregate([
#     {
#         "$match": {
#             "fields.description": {
#                 "$exists": True, 
#                 "$type": "string",
#                 "$regex": "As a"
#             },
#             "fields.issuetype.name": {
#                 "$in": mappedTypes
#             }
#         }
#     },
#     {
#         "$sample": {
#             "size": 10
#         }
#     }
# ])
results = collection.aggregate([
    {
        "$match": {
            "fields.description": {
                "$exists": True, 
                "$type": "string"
            },
            "fields.issuetype.name": {
                "$in": mappedTypes
            }
        }
    },
    {
        "$sample": {
            "size": 10
        }
    }
])

for document in results:
    tickets.append(document)
    print("Id: " + str(document['id']) + " / Description: " + document['fields']['description'])

User Story structure: 
Redhat: 13279134, 13254138, 13282377, 13405745, 14248927(has only the story)

#### Update See use_case_3.ipynb

# Ticket Creation

(Apache, 13066997) -> Status is Open, coulb be intersting 
(Jira, 284325) -> could be a good example for the Description completness Prompt!!
(Qt, 187366) -> intersting for Bug Report STructre!

In [ ]:
### Globals ###
LOG.reset()
JIRA = "Jira"
FOLDERNAME = "fewShotExamples"

In [ ]:
evo_df = PICKLE_LIB.pickle_load("./jiraEvolutions/load_evolution_dataframe(jiras=[_"+JIRA+"_])", 'gzip')

In [ ]:
sample_id = "1075133"
sample = evo_df[evo_df["issue_id"] == sample_id]
sample

In [ ]:
evolutionStep = 0
ticket = helper.createTicket(sample, evolutionStep)
ticket

# Ticket PreProcessing

In [ ]:
def preprocessTickets(ticket):
    
    ### Convert CreatedDate and ResolvedDate to datetime
    c_date = ticket['CreatedDate'].values[0]
    c_dt_obj = datetime.strptime(c_date, '%Y-%m-%dT%H:%M:%S.%f%z')
    ticket['CreatedDate'] = c_dt_obj.strftime('%Y-%m-%d %H:%M:%S')
    r_date = ticket['ResolvedDate'].values[0]
    r_dt_obj = datetime.strptime(r_date, '%Y-%m-%dT%H:%M:%S.%f%z')
    ticket['ResolvedDate'] = r_dt_obj.strftime('%Y-%m-%d %H:%M:%S')

    ### Convert IssueId to Int
    id = ticket['IssueId'].values[0]
    ticket['IssueId'] = int(id)

preprocessTickets(ticket)

In [ ]:
helper.saveTicket(FOLDERNAME, ticket, evolutionStep, JIRA, sample_id)

# Annotate Ticket

In [ ]:
def annotateTicket(ticket, annotation, reason):
    try:
        ticket['ViolationActual'] = annotation
        ticket['ViolationReason'] = reason
        print("Annotation successful.")
    except:
        print("Annotation failed.")

annotateTicket(ticket, "TRUE", """1. Resolution has to be set from 'None' to 'Low Priority'.
                           2. Status has to be set from 'Open' to 'Resolved'.""")

In [ ]:
if os.path.isfile("data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv"):
    dataset = pd.read_csv("./data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv")
    print("The dataset already exists.")
else:
    dataset = pd.DataFrame(columns=['Jira', 'IssueId', 'EvoId', 'Summary', 'Description', 'VersionsAffected', 'IssueType', 'Project', 'Components', 'CreatedDate', 'ResolvedDate', 'Status', 'Priority', 'Creator', 'Reporter', 'Resolution', 'IssueLinks', 'Labels','VersionsFixed', 'Assignee', 'TimeSpent', 'Comments', 'ViolationActual', 'ViolationReason', 'FieldCount', 'Fields'])
    print("The dataset was created successfully.")

dataset = pd.concat([dataset, ticket], ignore_index=True)

dataset.to_csv("data/" + FOLDERNAME + "/" + FOLDERNAME + "Dataset.csv", index=False)
print("The ticket was inserted into the dataset successfully!")


In [ ]:
dataset = pd.read_csv('data/' + FOLDERNAME + '/' + FOLDERNAME + 'Dataset.csv')
dataset